In [1]:
import pandas as pd

def average_court_time(data):
    try:
        if 'Duration' in data.columns and '__source_file__' in data.columns:
            data['Duration'] = pd.to_numeric(data['Duration'], errors='coerce')

            match_durations = (
                data.groupby('__source_file__')['Duration']
                .sum()
                .div(60)  # convert to minutes
                .round(1)
            )

            duration_minutes = match_durations.mean()
            if pd.notna(duration_minutes):
                hours = int(duration_minutes) // 60
                mins = int(duration_minutes) % 60
                return f"{hours}:{mins:02d}"

    except Exception as e:
        print(f"Error: {e}")

    return None

sets_df = pd.read_excel("/Users/eshaan/Desktop/Tennis Scouting/consulting-spring2025/data/mens/Rudy Quan/combined.xlsx", sheet_name="Sets")
final_duration = average_court_time(sets_df)
final_duration

'9:12'